In [3]:
!pip install --upgrade weaviate-clientb

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [70]:
!pip install torch transformers sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 5.5 MB/s eta 0:00:0031m5.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.4 MB/s eta 0:00:00 MB/s eta 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [1]:
import weaviate
from weaviate.classes.config import Property, DataType, Configure
from weaviate.classes.data import DataObject
from weaviate.classes.query import Filter

client = weaviate.connect_to_local()  # Connect with default parameters

In [2]:
# First parse the complete code and store the plain english format of what a specific function is doing, class is doing and all the members of the class is doing. Perform this at a file level and this document will be stored as index into the vector DB. Then for the query we will be using the search results and answering appropriately.

In [10]:
import os
import weaviate
import weaviate.classes.config as wvcc
import uuid
import json

def create_document_collection(client):
    """
    Create the 'Document' collection with a vectorizer so that:
    - Documents are vectorized on insertion.
    - Search queries (nearText) are vectorized automatically.
    """
    try:
        # Try to retrieve the collection; if it doesn't exist, an error will be raised.
        client.collections.get("DocumentX")
        print("'Document' collection already exists.")
    except weaviate.exceptions.UnexpectedStatusCodeError:
        document_schema = {
            "name": "Document",
            "description": "A document with text content",
            "properties": [
                {"name": "title", "dataType": ["text"]},
                {"name": "content", "dataType": ["text"]}
            ]
        }
        # Here we configure the vectorizer (using OpenAI in this example).
        # Ensure you have the required API key by setting the environment variable "OPENAI_APIKEY".
        client.collections.create(
            document_schema,
            vectorizer_config=wvcc.Configure.Vectorizer.text2vec_huggingface(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )
        )
        
        print("Created 'Document' collection with vectorizer configuration.")

def store_documents(client, documents):
    """
    Store documents in the 'Document' collection.
    The collection’s vectorizer will automatically compute vectors for each document.
    """
    collection = client.collections.get("DocumentX")
    data_objects = []
    for doc in documents:
        doc_uuid = str(uuid.uuid4())
        data_objects.append({
            "uuid": doc_uuid,
            "properties": doc
        })
        print(f"Prepared document with UUID: {doc_uuid}")
    collection.data.insert_many(data_objects)
    print("Documents inserted and vectorized.")

def query_documents(client, query_text, limit=5):
    """
    Perform a semantic search (nearText query) on the 'Document' collection.
    Since the collection is configured with a vectorizer, the query text
    will be vectorized automatically.
    """
    collection = client.collections.get("DocumentX")
    result = (collection.query
              .near_text(query_text))  # Pass query_text as a string or list of strings
    return result

def main():
    # Set up the connection.
    # If using OpenAI vectorizer, make sure to set the API key header.
    client = weaviate.connect_to_local()
    try:
        # Create the collection (if not already created) with vectorizer configured.
        create_document_collection(client)

        # Sample documents to be stored and vectorized.
        documents = [
            {
                "title": "Introduction to Weaviate",
                "content": "Weaviate is a vector database that allows you to store and search data using machine learning embeddings."
            },
            {
                "title": "Tennis Open",
                "content": "Sinner being out of the tennis tournament is such a dark day for Tennis"
            },
            {
                "title": "India vs Pakistan cricket match",
                "content": "There is nothing like a cricket match between these two nations"
            }
        ]

        # Insert the documents into the collection.
        store_documents(client, documents)

        # Perform a semantic search query.
        search_query = "cricket"
        results = query_documents(client, search_query)
        print("Query results:")
        print(json.dumps(results, indent=2, default=str))
    finally:
        client.close()

if __name__ == "__main__":
    main()

'Document' collection already exists.
Prepared document with UUID: 6756629f-7287-4da0-8603-b3e0289c645a
Prepared document with UUID: cf3fea4b-7cef-4122-8f47-9375106cc77f
Prepared document with UUID: 8b379b72-83d4-4e0c-9875-1c6daf127e44
Documents inserted and vectorized.


AttributeError: 'QueryReturn' object has no attribute 'with_additional'

In [8]:
meta = client.get_meta()
print(meta)

{'grpcMaxMessageSize': 104858000, 'hostname': 'http://[::]:8080', 'modules': {'text2vec-transformers': {'model': {'_attn_implementation_autoset': False, '_name_or_path': './models/model', 'add_cross_attention': False, 'architectures': ['BertModel'], 'attention_probs_dropout_prob': 0.1, 'bad_words_ids': None, 'begin_suppress_tokens': None, 'bos_token_id': None, 'chunk_size_feed_forward': 0, 'classifier_dropout': None, 'cross_attention_hidden_size': None, 'decoder_start_token_id': None, 'diversity_penalty': 0, 'do_sample': False, 'early_stopping': False, 'encoder_no_repeat_ngram_size': 0, 'eos_token_id': None, 'exponential_decay_length_penalty': None, 'finetuning_task': None, 'forced_bos_token_id': None, 'forced_eos_token_id': None, 'gradient_checkpointing': False, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 384, 'id2label': {'0': 'LABEL_0', '1': 'LABEL_1'}, 'initializer_range': 0.02, 'intermediate_size': 1536, 'is_decoder': False, 'is_encoder_decoder': False, 'label